In [0]:
job_dsc = spark.read.csv("/FileStore/tables/skill2vec_50K.csv.gz", header=False, inferSchema=True)

source_path = '/FileStore/tables/skill2vec_50K_csv.gz'

destination_path= '/FileStore/tables/skill2vec_50K.csv.gz'

dbutils.fs.mv ('/FileStore/tables/skill2vec_50K_csv.gz', '/FileStore/tables/skill2vec_50K.csv.gz')

file_location = job_dsc.inputFiles()[0]
print("File Location:", file_location)

File Location: dbfs:/FileStore/tables/skill2vec_50K.csv.gz


In [0]:
#Q1
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

job_dsc = spark.read.csv("/FileStore/tables/skill2vec_50K.csv.gz", header=False, inferSchema=True)

from pyspark.sql.functions import count

numdesc_job_dsc= job_dsc.distinct().count()
print(numdesc_job_dsc)

expected_numdesc = 50000
if numdesc_job_dsc == expected_numdesc:
    print(f"The number of job descriptions is as expected : {numdesc_job_dsc}")
else:
    print(f"The number of job descriptions is not as expected. Expected: {expected_numdesc}, Found: {numdesc_job_dsc}")

The number of job descriptions is as expected : 50000


In [0]:
#Q2
from pyspark . sql import functions as F
jobdsc_columns1 = job_dsc.columns[1:]
joblist1 = job_dsc . select (F.array(jobdsc_columns1).alias(" skills "))

#joblist1.show()
skills_df1 = joblist1.withColumn( " num_skills " , F . size ( " skills " ) )
#skills_df . show ()

non_null_skills_df1= joblist1.withColumn ( " not_null " , F . array_except ( " skills " , F.array( F.lit ( None ) ) ) )
#non_null_skills_df1.show()

non_null_data = non_null_skills_df1.select(" not_null ").rdd.flatMap(lambda x: x[0]).collect()
    
from collections import Counter
freqq1= Counter(non_null_data)
top_skills2= freqq1.most_common(10)
top_skills2_df= spark.createDataFrame(top_skills2, ["skill", "count"])
top_skills2_df.show()
display(top_skills2_df)

+--------------------+-----+
|               skill|count|
+--------------------+-----+
|                Java| 1911|
|          Javascript| 1770|
|               Sales| 1705|
|Business Development| 1545|
|    Web Technologies| 1313|
|Communication Skills| 1305|
|         development| 1238|
|           Marketing| 1184|
|             Finance| 1078|
|                HTML| 1067|
+--------------------+-----+



skill,count
Java,1911
Javascript,1770
Sales,1705
Business Development,1545
Web Technologies,1313
Communication Skills,1305
development,1238
Marketing,1184
Finance,1078
HTML,1067


Databricks visualization. Run in Databricks to view.

In [0]:
#Q3
from pyspark . sql import functions as F
jobdsc_columns = job_dsc.columns[1:]
joblist = job_dsc . select (F.array(jobdsc_columns).alias(" skills "))
#joblist.show()

skills_df = joblist.withColumn( " num_skills " , F . size ( " skills " ) )
#skills_df . show ()

non_null_skills_df= joblist.withColumn ( " not_null " , F . array_except ( " skills " , F.array( F.lit ( None ) ) ) )
#non_null_skills_df.show()

sizeDF = non_null_skills_df. withColumn ( " num_skills " , F . size ( F . col ( " not_null " ) ) )
#sizeDF.show()

count_freqq = sizeDF.groupBy(" num_skills ").count()
top_5_skillss = count_freqq.orderBy("count", ascending=False).limit(5)
top_5_skillss.show()

+------------+-----+
| num_skills |count|
+------------+-----+
|          10|10477|
|           5| 3432|
|           6| 3405|
|           1| 3386|
|           7| 3345|
+------------+-----+



In [0]:
#Q4
from pyspark . sql import functions as F
jobdsc_columns2 = job_dsc.columns[1:]
joblist2 = job_dsc . select (F.array(jobdsc_columns2).alias(" skills "))

#joblist2.show()
skills_df2 = joblist2.withColumn( " num_skills " , F . size ( " skills " ) )

#skills_df2 . show ()

non_null_skills_df2= joblist2.withColumn ( " not_null " , F . array_except ( " skills " , F.array( F.lit ( None ) ) ) )
non_null_data1 = non_null_skills_df2.select(" not_null ").rdd.flatMap(lambda x: x[0]).collect()

non_null_data1_lower= [data.lower() for data in non_null_data1]
frequency= Counter(non_null_data1_lower)

top= frequency.most_common(10)
top_skills_df = spark.createDataFrame(top, ["skill", "count"])
top_skills_df.show()
display(top_skills_df)

+--------------------+-----+
|               skill|count|
+--------------------+-----+
|                java| 2759|
|          javascript| 2738|
|               sales| 2680|
|business development| 2108|
|           marketing| 1809|
|                 sql| 1564|
|              jquery| 1547|
|                html| 1539|
|communication skills| 1537|
|                 bpo| 1530|
+--------------------+-----+



skill,count
java,2759
javascript,2738
sales,2680
business development,2108
marketing,1809
sql,1564
jquery,1547
html,1539
communication skills,1537
bpo,1530


Databricks visualization. Run in Databricks to view.

In [0]:
#Q5
non_null_data1_lower_c = len(non_null_data1_lower)
#print(non_null_data1_lower_c)

# Load O*NET dataset
onet= spark.read.option("delimiter", "\t").option("header", "true").option("inferSchema", "true").option("endOfLine", "\r\n").csv("/FileStore/tables/Technology_Skills.txt")

#Lowercase the skills
from pyspark.sql.functions import col
from pyspark.sql.functions import lower
onet_dfr = onet.select(*[lower(col('Example')).alias('Skill')] + [col(col_name) for col_name in onet.columns if col_name != 'Example'])

#print(onet_dfr.count())
#onet_dfr.show()

non_null_data1_lower_df= spark.createDataFrame([(skill,) for skill in non_null_data1_lower], ["skill"])

#Join the JDs dataset with the O*NET dataset on lowercase skill names
joined_dfr = non_null_data1_lower_df.join(onet_dfr, on=["skill"], how="inner")
joined_count1 = joined_dfr.count()

#Print the number of skills before and after join
print("Number of skills Before Join: {}".format(non_null_data1_lower_c))
print("Number of skills After Join: {}".format(joined_count1))

Number of skills Before Join: 463803
Number of skills After Join: 1101498


In [0]:
#Q6
#joined_dfr.show()
commodity_title_freq = joined_dfr.groupBy("Commodity Title").count()
commodity_titles = commodity_title_freq.select("Commodity Title", "count")
top_10_commodity_titles = commodity_titles.orderBy(col("count").desc()).limit(10)

top_10_commodity_titles.show(truncate=False)
display(top_10_commodity_titles)

+-------------------------------------------------+------+
|Commodity Title                                  |count |
+-------------------------------------------------+------+
|Object or component oriented development software|324521|
|Web platform development software                |298754|
|Operating system software                        |190926|
|Development environment software                 |53013 |
|Data base management system software             |44132 |
|Analytical or scientific software                |33552 |
|Web page creation and editing software           |31682 |
|Data base user interface and query software      |29436 |
|Spreadsheet software                             |18568 |
|File versioning software                         |13846 |
+-------------------------------------------------+------+



Commodity Title,count
Object or component oriented development software,324521
Web platform development software,298754
Operating system software,190926
Development environment software,53013
Data base management system software,44132
Analytical or scientific software,33552
Web page creation and editing software,31682
Data base user interface and query software,29436
Spreadsheet software,18568
File versioning software,13846


Databricks visualization. Run in Databricks to view.